In [2]:
import openai
import json

from tf.fabric import Fabric

from pathlib import Path

In [3]:
API_KEY_FILE = Path('/Users/cody/open_ai_api_key.txt')
API_KEY = API_KEY_FILE.read_text().strip()
openai.api_key = API_KEY

In [4]:
CORPUS = '/Users/cody/github/BH_time_collocations/data/data/corpus/'

tf_fabric = Fabric(CORPUS)
tf_api = tf_fabric.loadAll()

F, E, T, L = (getattr(tf_api, l) for l in 'FETL')

  1.03s Feature overview: 87 for nodes; 6 for edges; 1 configs; 9 computed


In [11]:
for clause in L.d(T.nodeFromSection(("1_Samuel", 9)), 'clause'):
    print(clause)
    print(T.text(clause))

330195
וַֽיְהִי־אִ֣ישׁ מִבִּנְיָמִ֗ין 
330196
וּ֠שְׁמֹו קִ֣ישׁ בֶּן־אֲבִיאֵ֞ל בֶּן־צְרֹ֧ור בֶּן־בְּכֹורַ֛ת בֶּן־אֲפִ֖יחַ 
330197
בֶּן־אִ֣ישׁ יְמִינִ֑י 
330198
גִּבֹּ֖ור חָֽיִל׃ 
330199
וְלֹו־הָיָ֨ה בֵ֜ן 
330200
וּשְׁמֹ֤ו שָׁאוּל֙ 
330201
בָּח֣וּר וָטֹ֔וב 
330202
וְאֵ֥ין אִ֛ישׁ מִבְּנֵ֥י יִשְׂרָאֵ֖ל טֹ֣וב מִמֶּ֑נּוּ 
330203
מִשִּׁכְמֹ֣ו וָמַ֔עְלָה גָּבֹ֖הַּ מִכָּל־הָעָֽם׃ 
330204
וַתֹּאבַ֨דְנָה֙ הָאֲתֹנֹ֔ות לְקִ֖ישׁ אֲבִ֣י שָׁא֑וּל 
330205
וַיֹּ֨אמֶר קִ֜ישׁ אֶל־שָׁא֣וּל בְּנֹ֗ו 
330206
קַח־נָ֤א אִתְּךָ֙ אֶת־אַחַ֣ד מֵֽהַנְּעָרִ֔ים 
330207
וְק֣וּם 
330208
לֵ֔ךְ 
330209
בַּקֵּ֖שׁ אֶת־הָאֲתֹנֹֽת׃ 
330210
וַיַּעֲבֹ֧ר בְּהַר־אֶפְרַ֛יִם 
330211
וַיַּעֲבֹ֥ר בְּאֶֽרֶץ־שָׁלִ֖שָׁה 
330212
וְלֹ֣א מָצָ֑אוּ 
330213
וַיַּעַבְר֤וּ בְאֶֽרֶץ־שַׁעֲלִים֙ 
330214
וָאַ֔יִן 
330215
וַיַּעֲבֹ֥ר בְּאֶֽרֶץ־יְמִינִ֖י 
330216
וְלֹ֥א מָצָֽאוּ׃ 
330217
הֵ֗מָּה בָּ֚אוּ בְּאֶ֣רֶץ צ֔וּף 
330218
וְשָׁא֥וּל אָמַ֛ר לְנַעֲרֹ֥ו 
330219
אֲשֶׁר־עִמֹּ֖ו 
330220
לְכָ֣ה 
330221
וְנָשׁ֑וּבָה 
330222
פֶּן־יֶחְדַּ֥ל אָבִ֛י מִן־הָאֲ

In [45]:
role_description = """

You are a linguistic analysis model that tags clauses with aspectual categories. Your specialization is in
Biblical Hebrew, and you are able to tag any clause from that corpus with advanced linguistic labels.
To arrive at your classifications, you will look carefully at the verb lexemes, verbal inflection, clause structure,
and valency of the clause.

There are 4 main types of aspectual types, following Vendler:

states: durative, non-dynamic event, unbounded (no inherent endpoint)
activities: durative, dynamic event, unbounded (no inherent endpoint)
achievements: punctual, dynamic event, bounded (an inherent endpoint is implied)
accomplishments: durative, dynamic event, bounded (inherent endpoint)

William Croft (2012, Verbs) adds aditional nuance to these categories, which you will use for labeling clauses.
Here are the expected labels you must assign, their description, and some example input and output:

transitory state:
    label: sta_tr
    description: Transitory states have a start and may have an end, represented by the ending of the transitory state before the end of the time dimension.
    English example: The door is open.
    example input: {"clause": "וַיָּ֥לֶן שָׁ֖ם בַּלַּ֣יְלָה הַה֑וּא ", "context": "וַיָּ֥לֶן שָׁ֖ם בַּלַּ֣יְלָה הַה֑וּא וַיִּקַּ֞ח מִן־הַבָּ֧א בְיָדֹ֛ו מִנְחָ֖ה לְעֵשָׂ֥ו אָחִֽיו׃ "}
    example output: {"aspect": "sta_tr"}

acquired state:
    label: sta_ac
    description: Acquired states are true of the entity for its entire lifetime once the state has been acquired.
    English example: The window is shattered.
    example input: {"clause": "כִּֽי־מָרָ֥ה תִהְיֶ֖ה בָּאַחֲרֹונָ֑ה ", "context": "וַיִּקְרָ֨א אַבְנֵ֜ר אֶל־יֹואָ֗ב וַיֹּ֨אמֶר֙ הֲלָנֶ֨צַח֙ תֹּ֣אכַל חֶ֔רֶב הֲלֹ֣וא יָדַ֔עְתָּה כִּֽי־מָרָ֥ה תִהְיֶ֖ה בָּאַחֲרֹונָ֑ה וְעַד־מָתַי֙ לֹֽא־תֹאמַ֣ר לָעָ֔ם לָשׁ֖וּב מֵאַחֲרֵ֥י אֲחֵיהֶֽם׃ "}
    example output: {"aspect": "sta_ac"}

inherent state:
    label: sta_in
    description: Inherent states are true of the entity for its entire lifetime.
    English example: She is French.
    example input: {"clause": "אֲשֶׁ֥ר יִהְיֶ֖ה בַּיָּמִ֣ים הָהֵ֑ם ", "context": "וְיָשַׁ֣ב׀ בָּעִ֣יר הַהִ֗יא עַד־עָמְדֹ֞ו לִפְנֵ֤י הָֽעֵדָה֙ לַמִּשְׁפָּ֔ט עַד־מֹות֙ הַכֹּהֵ֣ן הַגָּדֹ֔ול אֲשֶׁ֥ר יִהְיֶ֖ה בַּיָּמִ֣ים הָהֵ֑ם אָ֣ז׀ יָשׁ֣וּב הָרֹוצֵ֗חַ וּבָ֤א אֶל־עִירֹו֙ וְאֶל־בֵּיתֹ֔ו אֶל־הָעִ֖יר אֲשֶׁר־נָ֥ס מִשָּֽׁם׃ "}
    example output: {"aspect": "sta_in"}

point state:
    label: sta_po
    description: Point states such as (the sun) is at its zenith profile a state that lasts only a point in time. What counts as a point in time is also subject to construal. 
    English example:  The sun was at its zenith.
    example input: {"clause": "The sun was at its zenith.", "context": ""}
    example output: {"aspect": "sta_po"}

reversible directed achievement
    label: ach_rd
    description: Reversible achievements result in transitory, hence reversible, result states. This is a directed achievement, which is defined in the geometry as profiled transitions in which the entailed (unprofiled) result state phase is a state at a different point on the q dimension from the presupposed (unprofiled) initial rest state phase.
    English example: The door opened.
    example input: {"clause": "וַיַּשְׁכִּ֜ימוּ אַנְשֵׁ֤י הָעִיר֙ בַּבֹּ֔קֶר ", "context": "וַיַּשְׁכִּ֜ימוּ אַנְשֵׁ֤י הָעִיר֙ בַּבֹּ֔קֶר וְהִנֵּ֤ה נֻתַּץ֙ מִזְבַּ֣ח הַבַּ֔עַל וְהָאֲשֵׁרָ֥ה אֲשֶׁר־עָלָ֖יו כֹּרָ֑תָה וְאֵת֙ הַפָּ֣ר הַשֵּׁנִ֔י הֹֽעֲלָ֔ה עַל־הַמִּזְבֵּ֖חַ הַבָּנֽוּי׃ "}
    example output: {"aspect": "ach_rd"}

irreversible directed achievement
    label: ach_id
    description: Irreversible achievements result in permanent, hence irreversible, result states. This is a directed achievement, which is defined in the geometry as profiled transitions in which the entailed (unprofiled) result state phase is a state at a different point on the q dimension from the presupposed (unprofiled) initial rest state phase.
    English example: The window shattered.
    example input: {"clause": "וַיָּ֣מֶת׀ בַּיֹּ֣ום הַה֗וּא שְׁמֹנִ֤ים וַחֲמִשָּׁה֙ אִ֔ישׁ ", "context": "וַיֹּ֤אמֶר הַמֶּ֨לֶךְ֙ לְדֹואֵ֔ג סֹ֣ב אַתָּ֔ה וּפְגַ֖ע בַּכֹּהֲנִ֑ים וַיִּסֹּ֞ב דֹּואֵ֣ג הָאֲדֹמִ֗י וַיִּפְגַּע־הוּא֙ בַּכֹּ֣הֲנִ֔ים וַיָּ֣מֶת׀ בַּיֹּ֣ום הַה֗וּא שְׁמֹנִ֤ים וַחֲמִשָּׁה֙ אִ֔ישׁ נֹשֵׂ֖א אֵפֹ֥וד בָּֽד׃ "}
    example output: {"aspect": "ach_id"}

cyclic achievement
    label: ach_cy
    description: Cyclic achievements result in point states, which then revert to the rest state. For example, the mouse emits a squeak, which is a pointlike sound, and then is silent again (until the next squeak)
    English example: The mouse squeaked.
    example input: {"clause": "וַיִּקְרָ֧א אֶל־מֹשֶׁ֛ה בַּיֹּ֥ום הַשְּׁבִיעִ֖י מִתֹּ֥וךְ הֶעָנָֽן׃ ", "context": "וַיִּשְׁכֹּ֤ן כְּבֹוד־יְהוָה֙ עַל־הַ֣ר סִינַ֔י וַיְכַסֵּ֥הוּ הֶעָנָ֖ן שֵׁ֣שֶׁת יָמִ֑ים וַיִּקְרָ֧א אֶל־מֹשֶׁ֛ה בַּיֹּ֥ום הַשְּׁבִיעִ֖י מִתֹּ֥וךְ הֶעָנָֽן׃ "}
    example output: {"aspect": "ach_cy"}

directed activity
    label: act_di
    description: Directed activities involve a continuous (or at least incremental) change along the q dimension, but without a transition to a result state representing a completed action (this will be described shortly with accomplishments).
    English example: The soup cooled [for a while].
    example input: {"clause": "וַיִּסְע֤וּ מֵֽרַעְמְסֵס֙ בַּחֹ֣דֶשׁ הָֽרִאשֹׁ֔ון בַּחֲמִשָּׁ֥ה עָשָׂ֛ר יֹ֖ום לַחֹ֣דֶשׁ הָרִאשֹׁ֑ון ", "context": "וַיִּסְע֤וּ מֵֽרַעְמְסֵס֙ בַּחֹ֣דֶשׁ הָֽרִאשֹׁ֔ון בַּחֲמִשָּׁ֥ה עָשָׂ֛ר יֹ֖ום לַחֹ֣דֶשׁ הָרִאשֹׁ֑ון מִֽמָּחֳרַ֣ת הַפֶּ֗סַח יָצְא֤וּ בְנֵֽי־יִשְׂרָאֵל֙ בְּיָ֣ד רָמָ֔ה לְעֵינֵ֖י כָּל־מִצְרָֽיִם׃ "}
    example output: {"aspect": "act_di"}

undirected activity
    label: act_un
    description: Undirected activities do not involve a continuous directed change along qualitative states in the q dimension.
    English example: The girls chanted.
    example input: {"clause": "וָֽאֶתְפַּלֵּ֛ל גַּם־בְּעַ֥ד אַהֲרֹ֖ן בָּעֵ֥ת הַהִֽוא׃ ", "context": "וּֽבְאַהֲרֹ֗ן הִתְאַנַּ֧ף יְהוָ֛ה מְאֹ֖ד לְהַשְׁמִידֹ֑ו וָֽאֶתְפַּלֵּ֛ל גַּם־בְּעַ֥ד אַהֲרֹ֖ן בָּעֵ֥ת הַהִֽוא׃ "}
    example output: {"aspect": "act_un"}

incremental accomplishment
    label: acc_in
    description: Three phases are profiled: the inception and completion phase as well as the directed change phase. This is because accomplishments are durative but bounded: the inception and completion phases bound the accomplishment. Accomplishments such as write a letter are said to have a ‘natural endpoint’; this natural endpoint is the result state on the q dimension, and progression toward the natural endpoint is defined by the points of the scale of the directed change
    English example: I ate an apple pancake.
    example input: {"clause": "וַיִּכְתֹּ֥ב מֹשֶׁ֛ה אֶת־הַשִּׁירָ֥ה הַזֹּ֖את בַּיֹּ֣ום הַה֑וּא ", "context": "וַיִּכְתֹּ֥ב מֹשֶׁ֛ה אֶת־הַשִּׁירָ֥ה הַזֹּ֖את בַּיֹּ֣ום הַה֑וּא וַֽיְלַמְּדָ֖הּ אֶת־בְּנֵ֥י יִשְׂרָאֵֽל׃ "}
    example output: {"aspect": "acc_in"}

runup accomplishment
    label: acc_ru
    description: Three phases are profiled: the inception and completion phase as well as the directed change phase. This is because accomplishments are durative but bounded: the inception and completion phases bound the accomplishment. A runup achievement is represented as a bounded event with profiled begin- ning and end transition phases, but the middle profiled phase is an undirected activity.
    English example: Harry repaired the computer.
    example input: {"clause": "וַיַּעֲשׂ֣וּ אֶת־הַפֶּ֡סַח בְּאַרְבָּעָה֩ עָשָׂ֨ר יֹ֥ום לַחֹ֛דֶשׁ בָּעֶ֖רֶב בְּעַֽרְבֹ֥ות יְרִיחֹֽו׃ ", "context": "וַיַּחֲנ֥וּ בְנֵֽי־יִשְׂרָאֵ֖ל בַּגִּלְגָּ֑ל וַיַּעֲשׂ֣וּ אֶת־הַפֶּ֡סַח בְּאַרְבָּעָה֩ עָשָׂ֨ר יֹ֥ום לַחֹ֛דֶשׁ בָּעֶ֖רֶב בְּעַֽרְבֹ֥ות יְרִיחֹֽו׃ "}
    example output: {"aspect": "acc_ru"}

""".strip()

role_description_shortened = """
You are a linguistic analysis model specializing in Biblical Hebrew. Your task is to tag clauses from the corpus with aspectual categories using advanced linguistic labels.
Here are the labels you should use:
1. Transitory State:
   - Label: sta_tr
   - English example: The door is open.
   - Example input: {"clause": "וַיָּ֥לֶן שָׁ֖ם בַּלַּ֣יְלָה הַה֑וּא ", "context": "וַיָּ֥לֶן שָׁ֖ם בַּלַּ֣יְלָה הַה֑וּא וַיִּקַּ֞ח מִן־הַבָּ֧א בְיָדֹ֛ו מִנְחָ֖ה לְעֵשָׂ֥ו אָחִֽיו׃ "}
2. Acquired State:
   - Label: sta_ac
   - English example: The window is shattered.
   - Example input: {"clause": "כִּֽי־מָרָ֥ה תִהְיֶ֖ה בָּאַחֲרֹונָ֑ה ", "context": "וַיִּקְרָ֨א אַבְנֵ֜ר אֶל־יֹואָ֗ב וַיֹּ֨אמֶר֙ הֲלָנֶ֨צַח֙ תֹּ֣אכַל חֶ֔רֶב הֲלֹ֣וא יָדַ֔עְתָּה כִּֽי־מָרָ֥ה תִהְיֶ֖ה בָּאַחֲרֹונָ֑ה ו
ְעַד־מָתַי֙ לֹֽא־תֹאמַ֣ר לָעָ֔ם לָשׁ֖וּב מֵאַחֲרֵ֥י אֲחֵיהֶֽם׃ "}
3. Inherent State:
   - Label: sta_in
   - English example: She is French.
   - Example input: {"clause": "אֲשֶׁ֥ר יִהְיֶ֖ה בַּיָּמִ֣ים הָהֵ֑ם ", "context": "וְיָשַׁ֣ב׀ בָּעִ֣יר הַהִ֗יא עַד־עָמְדֹ֞ו לִפְנֵ֤י הָֽעֵדָה֙ לַמִּשְׁפָּ֔ט עַד־מֹות֙ הַכֹּהֵ֣ן הַגָּדֹ֔ול אֲשֶׁ֥ר יִהְיֶ֖ה בַּיָּמִ֣ים הָהֵ֑ם אָ֣ז׀ יָשׁ֣וּב הָרֹוצֵ֗חַ וּבָ֤א אֶל־עִירֹו֙ וְאֶל־בֵּיתֹ֔ו אֶל־הָעִ֖יר אֲשֶׁר־נָ֥ס מִשָּֽׁם׃ "}
4. Point State:
   - Label: sta_po
   - English example: The sun was at its zenith.
   - Example input: {"clause": "The sun was at its zenith.", "context": ""}
5. Reversible Directed Achievement:
   - Label: ach_rd
   - English example: The door opened.
   - Example input: {"clause": "וַיַּשְׁכִּ֜ימוּ אַנְשֵׁ֤י הָעִיר֙ בַּבֹּ
֔קֶר ", "context": "וַיַּשְׁכִּ֜ימוּ אַנְשֵׁ֤י הָעִיר֙ בַּבֹּ֔קֶר וְהִנֵּ֤ה נֻתַּץ֙ מִזְבַּ֣ח הַבַּ֔עַל וְהָאֲשֵׁרָ֥ה אֲשֶׁר־עָלָ֖יו כֹּרָ֑תָה וְאֵת֙ הַפָּ֣ר הַשֵּׁנִ֔י הֹֽעֲלָ֔ה עַל־הַמִּזְבֵּ֖חַ הַבָּנֽוּי׃ "}
6. Irreversible Directed Achievement:
   - Label: ach_id
   - English example: The window shattered.
   - Example input: {"clause": "וַיָּ֣מֶת׀ בַּיֹּ֣ום הַה֗וּא שְׁמֹנִ֤ים וַחֲמִשָּׁה֙ אִ֔ישׁ ", "context": "וַיֹּ֤אמֶר הַמֶּ֨לֶךְ֙ לְדֹואֵ֔ג סֹ֣ב אַתָּ֔ה וּפְגַ֖ע בַּכֹּהֲנִ֑ים וַיִּסֹּ֞ב דֹּואֵ֣ג הָאֲדֹמִ֗י וַיִּפְגַּע־הוּא֙ בַּכֹּ֣הֲנִ֔ים וַיָּ֣מֶת׀ בַּיֹּ֣ום הַה֗וּא שְׁמֹנִ֤ים וַחֲמִשָּׁה֙ אִ֔ישׁ נֹשֵׂ
֖א אֵפֹ֥וד בָּֽד׃ "}
7. Cyclic Achievement:
   - Label: ach_cy
   - English example: The mouse squeaked.
   - Example input: {"clause": "וַיִּקְרָ֧א אֶל־מֹשֶׁ֛ה בַּיֹּ֥ום הַשְּׁבִיעִ֖י מִתֹּ֥וךְ הֶעָנָֽן׃ ", "context": "וַיִּשְׁכֹּ֤ן כְּבֹוד־יְהוָה֙ עַל־הַ֣ר סִינַ֔י וַיְכַסֵּ֥הוּ הֶעָנָ֖ן שֵׁ֣שֶׁת יָמִ֑ים וַיִּקְרָ֧א אֶל־מֹשֶׁ֛ה בַּיֹּ֥ום הַשְּׁבִיעִ֖י מִתֹּ֥וךְ הֶעָנָֽן׃ "}
8. Directed Activity:
   - Label: act_di
   - English example: The soup cooled [for a while].
   - Example input: {"clause": "וַיִּסְע֤וּ מֵֽרַעְמְסֵס֙ בַּחֹ֣דֶשׁ הָֽרִאשֹׁ֔ון בַּחֲמִשָּׁ֥ה עָשָׂ֛ר יֹ֖ום לַחֹ֣דֶשׁ הָרִאשֹׁ֑ון ", "context": "וַיִּסְע֤וּ מֵֽרַעְמְסֵס֙ בַּחֹ֣דֶשׁ הָֽרִאשֹׁ֔ון בַּחֲמִשָּׁ֥ה
 עָשָׂ֛ר יֹ֖ום לַחֹ֣דֶשׁ הָרִאשֹׁ֑ון מִֽמָּחֳרַ֣ת הַפֶּ֗סַח יָצְא֤וּ בְנֵֽי־יִשְׂרָאֵל֙ בְּיָ֣ד רָמָ֔ה לְעֵינֵ֖י כָּל־מִצְרָֽיִם׃ "}
9. Undirected Activity:
   - Label: act_un
   - English example: The girls chanted.
   - Example input: {"clause": "וָֽאֶתְפַּלֵּ֛ל גַּם־בְּעַ֥ד אַהֲרֹ֖ן בָּעֵ֥ת הַהִֽוא׃ ", "context": "וּֽבְאַהֲרֹ֗ן הִתְאַנַּ֧ף יְהוָ֛ה מְאֹ֖ד לְהַשְׁמִידֹ֑ו וָֽאֶתְפַּלֵּ֛ל גַּם־בְּעַ֥ד אַהֲרֹ֖ן בָּעֵ֥ת הַהִֽוא׃ "}
10. Incremental Accomplishment:
    - Label: acc_in
    - English example: I ate an apple pancake.
    - Example input: {"clause": "וַיִּכְתֹּ֥ב מֹשֶׁ֛ה אֶת־הַשִּׁירָ֥ה הַזֹּ֖את בַּיֹּ֣ום הַה֑וּא ", "context": "וַיִּכְתֹּ֥ב מֹשֶׁ֛ה אֶת־הַשִּׁירָ֥ה הַזֹּ֖את בַ
ּיֹּ֣ום הַה֑וּא וַֽיְלַמְּדָ֖הּ אֶת־בְּנֵ֥י יִשְׂרָאֵֽל׃ "}
11. Runup Accomplishment:
    - Label: acc_ru
    - English example: Harry repaired the computer.
    - Example input: {"clause": "וַיַּעֲשׂ֣וּ אֶת־הַפֶּ֡סַח בְּאַרְבָּעָה֩ עָשָּׁ֥ר יֹ֥ום לַחֹ֛דֶשׁ בָּעֶ֖רֶב בְּעַֽרְבֹ֥ות יְרִיחֹֽו׃ ", "context": "וַיַּחֲנ֥וּ בְנֵֽי־יִשְׂרָאֵ֖ל בַּגִּלְגָּ֑ל וַיַּעֲשׂ֣וּ אֶת־הַפֶּ֡סַח בְּאַרְבָּעָה֩ עָשָּׁ֥ר יֹ֥ום לַחֹ֛דֶשׁ בָּעֶ֖רֶב בְּעַֽרְבֹ֥ות יְרִיחֹֽו׃ "}

""".strip()

In [58]:
stop = 20

for i, clause in enumerate(F.kind.s('VC'), 1):
    print(clause)
    print(T.text(clause))
    print(T.text(L.u(clause, 'verse')[0]))
    print()
    if i == stop:
        break

302967
בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 
בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 

302968
וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ 
וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ 

302970
וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ 
וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ 

302971
וַיֹּ֥אמֶר אֱלֹהִ֖ים 
וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽור׃ 

302972
יְהִ֣י אֹ֑ור 
וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽור׃ 

302973
וַֽיְהִי־אֹֽור׃ 
וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽור׃ 

302974
וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור 
וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ 

302975
כִּי־טֹ֑וב 
וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ 

302976
וַיּ

In [59]:
messages = [
    {"role": "system", "content": role_description_shortened},
    {"role": "user", "content": '{"clause": "וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה ", "context": "וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום אֶחָֽד׃ פ "}'},
]

In [60]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

In [61]:
print(completion.choices[0].message.content)

Label: sta_in 

Translation: And to the darkness He called "night".
